## Week4
#### 그냥 인터넷에서 본 이것저것

tensor 2 부터 지원되는 tf.gradients는 graph로 표현된 층으로부터의 미분값들을 구한다.<br>
궁금해서 tf.graph를 찾아봤는데, 이는 파이썬 내에서만의 operation으로 구성된 기존 레이어들에서 벗어나,
graph 형식의 data structure를 구현하여 파이썬 interpreter가 없는 embeded, mobile, server 등에서 tf를 활용할 수 있도록 한다고 한다.

### 하긴
현재 보고 있는 강의도 그렇다. 기본적인 원리를 설명하기 위해 numpy로 직접 하나하나 구현하는 operation이지만, 이것은 딥러닝 프레임워크가 나옴에 따라 구조화 되어 우리가 편히 사용하는 것이니까. <br>
우선적으로 원리를 알기 위해 이 강의를 보고 있음을 잊지 말자.
----
hidden layer의 weight를 그림으로 나타내면, 서로 다른 빗면으로 나타나는 그림 여러장을 구할 수 있는데, 이는 DNN에서 주어진 iuput image에서 각 빗면이 어디에 위치하는지를 구하는 것이라고 한다.(아마 특이한 케이스일 것이다. 이미 다 세팅된..)<br>

그리고 layer를 지날때마다 x의 사이즈가 줄어들면서 눈, 코, 입 단위의 프레임이 턱, 이마, 안면 등으로 확대되며 학습이 진행된다. <br>
즉, 작은 부분을 찾아서 그들을 합쳐가며 복잡한 부분을 찾을 수 있는 것이라고..

마지막에, 작은 모델부터 시작해서 (logistic regression) 문제를 해결할 수 있는 모델을 찾는 것이 효율적이라고 하셨다.<br>
(물론, 문제에 따라 접근해야겟지만..) 이런 응용력이 기반되어야 할듯하다.

**loss가 구해짐에 따라**

backpropagation이 가능해지며 역으로 전파하고, 현재 loss 공식에 맞는 derivative term이 구해지고 전파되며 derivative weight가 구해짐

어쨌든.. 오늘은 DNN의 backpropagation을 배웠음.

In [5]:
import tensorflow.keras.layers as Layer
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [6]:
import os
import pickle

x = open('/root/storage/DATA/pickle/checked','rb')
data = pickle.load(x)
x.close()

cycle = np.random.permutation(25)
exp_data = data.reshape(-1,data.shape[1],data.shape[-1]*data.shape[-2])

In [15]:
def build(shape):
    x_input = Layer.Input(shape)
    
    x = Layer.Dense(100,activation='tanh')(x_input)
    x = Layer.Dense(50,activation='tanh')(x)
    x = Layer.Dense(25,activation='tanh')(x)
    
    model = Model(inputs=x_input,outputs=x,name='custom')
    return model
#x = np.random.randn(7,7760)

## 일단 너무 추우니까 철수 ;

---

학습을 하기 위해 loss를 구하는 식을 짜야하는데,, 우선 대상을 정하고 그 사이에서 식을 정립해야할듯

In [299]:
exp_shape = exp_data[0,0].shape
print(exp_shape)
model = build(exp_shape)
def calc_euclidian_dists(x, y):
    """
    Calculate euclidian distance between two 3D tensors.
    Args:
        x (tf.Tensor):
        y (tf.Tensor):
    Returns (tf.Tensor): 2-dim tensor with distances.
    """
    return tf.reduce_mean(tf.math.pow(x - y, 2), 1)

optimizer = tf.optimizers.Adam(10e-2)
iteration = 20
iper_cls = 5
with tf.device('/CPU:0'):
    for i in range(iteration):
        permu = np.random.permutation(25)[:iper_cls]
        data_pool = exp_data[permu].reshape(iper_cls*exp_data.shape[1],exp_data.shape[-1])
        #x_data = data_pool[:,:20]
        #y_data = data_pool[:,20:]
        with tf.GradientTape() as tape:
            result = model(data_pool)
            x_result = result[:100]
            y_result = result[100:]
            
            y = np.tile(np.arange(5)[:, np.newaxis], (1, 180))
            y_onehot = tf.cast(tf.expand_dims(y,axis=2), tf.float32)
            
            x_result=tf.tile(x_result,(y_result.shape[0]//x_result.shape[0],1))
            
            afc = calc_euclidian_dists(x_result,y_result)
            
            log_p_y = tf.nn.log_softmax(-afc, axis=-1)
            log_p_y = tf.reshape(log_p_y, [iper_cls, 180, -1])
            
            loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_onehot, log_p_y), axis=-1), [-1]))
        #print(model.trainable_weights)
        gradients = tape.gradient(loss,model.trainable_weights)
        print('iter :',i, loss)
        #print('grad : ',gradients)
        #print('weight : ',model.trainable_weights)
        
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        


(7760,)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

iter : 0 tf.Tensor(13.665147, shape=(), dtype=float32)
iter : 1 tf.Tensor(13.699387, shape=(), dtype=float32)
iter : 2 tf.Tensor(13.916614, shape=(), dtype=float32)
iter : 3 tf.Tensor(14.426247, shape=(), dtype=float32)
iter : 4 tf.Tensor(13.777717, shape=(), dtype=float32)
iter : 5 tf.Tensor(13.799385, shape=(), dtype=float32)
iter : 6 tf.Tensor(14.059562, shape=(), dtype=float32)
iter : 7 tf.Tensor(13.602678, shape=(), dtype=float32)
iter : 8 tf.Tensor(13.630167, shape=(), dtype=float32)
iter 